<a href="https://colab.research.google.com/github/yadavrishikesh/st.using.BKTR/blob/main/EcoCounter_DailyBike_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages("BKTR")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘coro’, ‘safetensors’, ‘collections’, ‘png’, ‘plyr’, ‘jpeg’, ‘bitops’, ‘torch’, ‘R6P’, ‘ggmap’




In [ ]:
data("EcoCounter_bike_data")

## spatial info
spatial_positions_df<- data.table(location = 1:dim(daily_bike_data$loc)[1],
                                  latitude = daily_bike_data$loc[,"lat"],
                                  longitude = daily_bike_data$loc[,"long"]
                                  )
spatial_positions_df<- setkey(spatial_positions_df, location)
ns<- nrow(spatial_positions_df)

## temporal info
temporal_positions_df<- data.table(time = c(seq(from = as.Date("2021-04-15"), to = as.Date("2021-11-15"), by = "day"),
                                            seq(from = as.Date("2022-04-15"), to = as.Date("2022-11-15"), by = "day")),
                                   time_index  = 0: (nrow(daily_bike_data$bikecounts)-1)
                                   )
temporal_positions_df<- setkey(temporal_positions_df, time)
nt<- nrow(temporal_positions_df)


#### data
Y.all<- daily_bike_data$bikecounts
miss.artfical.na<-  list(site1 = 1:20, site15 = 200:230,
                         site25 = 350:400, site40 = 100:150)
#### artificially creating some NAs to asses the missing value imputations performances
Y<- Y.all
Y[miss.artfical.na$site1, 1]<- NA
Y[miss.artfical.na$site15, 15]<- NA
Y[miss.artfical.na$site25, 25]<- NA
Y[miss.artfical.na$site40, 40]<- NA

## whole datasets
normalize <- function(x) {
  return ((x - min(x)) / (max(x) - min(x)))
}
data_df<- data.table(location  = rep(spatial_positions_df$location, each=nt),
                     time  = rep(temporal_positions_df$time, ns),
                     log.counts = c(log(1+Y)),
                     temp = rep(normalize(daily_bike_data$purely_temp_cov$temp), ns),
                     visb = rep(normalize(daily_bike_data$purely_temp_cov$visbl), ns),
                     wspd = rep(normalize(daily_bike_data$purely_temp_cov$wsp), ns),
                     precp.dummy = rep(normalize(daily_bike_data$purely_temp_cov$precp.dummy), ns),
                     year.dummy = rep(normalize(daily_bike_data$purely_temp_cov$year.dummy), ns),
                     sin7 =  sin((2*pi*(1:nt))/7),
                     cos7 =  cos((2*pi*(1:nt))/7),
                     elevation = rep(normalize(daily_bike_data$purely_spatial_cov$elev), each = nt),
                     walkscore = rep(normalize(daily_bike_data$purely_spatial_cov$walkscore), each = nt),
                     num_pop = rep(normalize(daily_bike_data$purely_spatial_cov$num_ppo), each = nt)
)
data_df<- setkey(data_df, location, time)
dim(data_df)
ns * nt


In [ ]:
############## predictions (spatial interpolation) at new sites #########################
spatInt.ind = c(11, 34, 41, 44)
new_s<- spatial_positions_df$location[spatInt.ind]  ###c(100003032, 100041101, 300020679)
forcast.ind<- 1:7
new_t<- temporal_positions_df$time[-(1:(nt-length(forcast.ind)))]
new_t <- as.IDate(new_t) ##Cast to IDate to match implicit cast of data.table
obs_s <- setdiff(unlist(spatial_positions_df$location), new_s)
obs_t <- as.IDate(setdiff(unlist(temporal_positions_df$time), new_t))


### obs_data_df$log.counts<- c(data_mat) ## in case of imputations
obs_data_df <- data_df[data_df[, .I[location %in% obs_s & time %in% obs_t]], ] ### in case of no
obs_spa_df <- spatial_positions_df[spatial_positions_df[, .I[location %in% obs_s]], ]
obs_tem_df <- temporal_positions_df[temporal_positions_df[,.I[time %in% obs_t]], ]

In [ ]:
# Train and predict
TSR$set_params(seed = 1) # Set the seed
k_local_periodic <- KernelSE$new() *
  KernelPeriodic$new(period_length = KernelParameter$new(value = 7, is_fixed = TRUE))
#formula.bktr<- log.counts ~ 1 + temp + visb + wspd + prep + elevation + walkscore  + num_pop + sin7 + cos7
formula.bktr<- log.counts ~ 1 + temp + visb  + wspd + precp.dummy + year.dummy + sin7 + cos7  +  elevation + walkscore + num_pop

BKTR::TSR$set_params(fp_type = 'float64')
bktr_regressor <- BKTRRegressor$new(
  formula = formula.bktr ,
  data_df=obs_data_df,
  spatial_positions_df=obs_spa_df,
  temporal_positions_df=obs_tem_df,
  burn_in_iter=50, ### by default choice is 1000
  sampling_iter = 50, ## by default choice is 500
  rank = 10, ## by default choice is 10
  spatial_kernel = KernelMatern$new(smoothness_factor = 1), # default spatial kernel is Matern 5/2, i.e., with nu=5
  temporal_kernel = k_local_periodic # default temporal kernel is squired exponential
)
bktr_regressor$mcmc_sampling()
summary(bktr_regressor)

In [ ]:
plot_y_estimates(bktr_regressor)

In [ ]:
### predictions
new_data_df<- data_df[data_df[, .I[location %in% new_s | time %in% new_t]], ]
new_spa_df <- spatial_positions_df[spatial_positions_df[, .I[location %in% new_s]], ]
new_tem_df <- temporal_positions_df[temporal_positions_df[, .I[time %in% new_t]], ]
preds <- bktr_regressor$predict(new_data_df, new_spa_df, new_tem_df)


In [ ]:

## overall- predictions
par(mfrow=c(1,1))
plot(preds$new_y_df$y_est, new_data_df$log.counts)


In [ ]:
############## predictions (spatial interpolation) at new sites #########################
#new_data_df <- data_df[data_df[, .I[location %in% new_s | time %in% new_t]], c('location', 'time', 'log.counts')]
pred_y_df <- preds$new_y_df
setkey(new_data_df, location, time)
setkey(pred_y_df, location, time)
non_na_indices <- which(!is.na(new_data_df$log.counts))

In [ ]:

### reporting the estimated values
spatInt.ind.sorted <- sort(spatInt.ind)
n_sites <- length(spatInt.ind.sorted)
n_t <- length(new_t)
true.forecast.counts <- list()
true.spatial.intpl.counts <- list()
true.st.pred.counts <- list()
est.forecast.counts <- list()
est.spatial.intpl.counts <- list()
est.st.pred.counts <- list()
start_idx <- 1

for (i in 1:n_sites) {
  if (i == 1) {
    shift <- spatInt.ind.sorted[i] - 1
    if (spatInt.ind.sorted[i] == 1) shift <- 1
  } else {
    shift <- spatInt.ind.sorted[i] - spatInt.ind.sorted[i-1] - 1
    if (spatInt.ind.sorted[i] - spatInt.ind.sorted[i-1] == 1) shift <- 1
  }

  if (shift < 1) next
  true.forecast.counts[[i]] <- matrix(new_data_df$log.counts[start_idx : (start_idx + n_t * shift - 1)], nrow = n_t, ncol = shift)
  true.spatial.intpl.counts[[i]] <- new_data_df$log.counts[(start_idx + n_t * shift) : (start_idx + n_t * shift + (nt-n_t)-1)]
  true.st.pred.counts[[i]] <- new_data_df$log.counts[(start_idx + n_t * shift + (nt-n_t)) : (start_idx + n_t * shift + nt-1)]

  est.forecast.counts[[i]] <- matrix(pred_y_df$y_est[start_idx : (start_idx + n_t * shift - 1)], nrow = n_t, ncol = shift)
  est.spatial.intpl.counts[[i]] <- pred_y_df$y_est[(start_idx + n_t * shift) : (start_idx + n_t * shift + (nt-n_t) -1)]
  est.st.pred.counts[[i]] <- pred_y_df$y_est[(start_idx + n_t * shift + (nt-n_t)) : (start_idx + n_t * shift + nt - 1)]

  start_idx <- start_idx + n_t * shift + nt
}

forecast <- list(true = exp(do.call(cbind, true.forecast.counts)), est.mean = exp(do.call(cbind, est.forecast.counts)))
spat_time_pred <- list(true = exp(do.call(cbind, true.st.pred.counts)), est.mean = exp(do.call(cbind, est.st.pred.counts)))
spat.interpol <- list(true = exp(do.call(cbind, true.spatial.intpl.counts)), est.mean = exp(do.call(cbind, est.spatial.intpl.counts)))
outputs.CVs<- list("spat.interpol." = spat.interpol, "forecast" = forecast, "space.time.pred" =  spat_time_pred)


In [ ]:
par(mfrow=c(3,1))
plot(c(outputs.CVs$spat.interpol.$true), c(outputs.CVs$spat.interpol.$est.mean),
     xlab="true", ylab = "est")
plot(c(outputs.CVs$forecast$true), c(outputs.CVs$forecast$est.mean),
    xlab="true", ylab = "est")
plot(c(outputs.CVs$space.time.pred$true), c(outputs.CVs$space.time.pred$est.mean),
    xlab="true", ylab = "est")